In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import unidecode
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
import requests
from bs4 import BeautifulSoup

In [2]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [3]:
def obtain_links_periodos(driver):
    links_asistencias = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('votaciones', link)):
            links_asistencias.append(link)
    return(links_asistencias)

def obtain_links_votaciones(driver):
    links_asistencias_sesion = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('estadistico_votacion', link)):
            links_asistencias_sesion.append(link)
    return(links_asistencias_sesion)

def obtain_partyVotaciones(driver):
    links_asistencias_sesion = []
    partidos = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('listados_votaciones', link)):
            links_asistencias_sesion.append(link)
            partidos.append(i.text)
    return(links_asistencias_sesion, partidos)

In [5]:

r = requests.get('http://sitllx.diputados.gob.mx/votaciones_por_periodonp.php')
soup = BeautifulSoup(r.content, 'html.parser')

In [120]:
def obtain_topic(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    topic = re.sub('\n+', '', soup.findAll('table')[0].text.split('Grupo Parlamentario')[0]).split('\xa0')[1]
    return(topic)

In [37]:
def month_to_num(x):
    if x == 'Enero':
        return(1)
    elif x == 'Febrero':
        return(2)
    elif x == 'Marzo':
        return(3)
    elif x == 'Abril':
        return(4)
    elif x == 'Mayo':
        return(5)
    elif x == 'Junio':
        return(6)
    elif x == 'Julio':
        return(7)
    elif x == 'Agosto':
        return(8)
    elif x == 'Septiembre':
        return(9)
    elif x == 'Octubre':
        return(10)
    elif x == 'Noviembre':
        return(11)
    elif x == 'Diciembre':
        return(12)

In [112]:
def obtain_votacion(link, legislatura, partido, topic):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    temp = re.findall('[0-9]+[, -][A-Z][a-z]+[-][0-9]+', soup.findAll('tr')[4].text)[0]
    date_session = datetime.date(int(temp.split('-')[2]), month_to_num(temp.split('-')[1]), int(temp.split('-')[0]))
    party = re.sub('\n', '', soup.findAll('tr')[7].text)
    
    year_legis = []
    month_legis = []
    day_legis = []
    members = []
    party = []
    sentido_votacion = []
    legis = []
    titulo = []
    for i in soup.findAll('tr')[10:]:
        try:
            if any(re.findall('[a-z]+', i.findAll('td')[0].text)):
                continue
            else:
                members.append(unidecode.unidecode(i.findAll('td')[1].text))
                sentido_votacion.append(i.findAll('td')[2].text)
                titulo.append(topic)
                party.append(partido)
                year_legis.append(date_session.year)
                month_legis.append(date_session.month)
                day_legis.append(date_session.day)
                legis.append(legislatura)
        except:
            continue
            
    return(pd.DataFrame({'legislatura':legis, 
                 'year_session':year_legis, 
                 'month_session':month_legis, 
                 'day_session':day_legis, 
                 'member':members, 
                 'party':party, 
                 'voto':sentido_votacion,
                         'titulo_votacion':titulo
                 }))



In [135]:
def obtain_links_votaciones(main_link):
    r = requests.get(main_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    links_asistencias_sesion = []
    for i in soup.findAll("table")[0].findAll('a'):
        link = i.get('href')
        if any(re.findall('estadistico_votacion', link)):
            links_asistencias_sesion.append(main_link.split('.gob.mx')[0] + '.gob.mx'+'/' + link)
    return(links_asistencias_sesion)

def obtain_partyVotaciones(main_link):
    r = requests.get(main_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    links_asistencias_sesion = []
    partidos = []
    for i in soup.findAll("table")[0].findAll('a'):
        link = i.get('href')
        if any(re.findall('listados_votaciones', link)):
            links_asistencias_sesion.append(main_link.split('.gob.mx')[0] + '.gob.mx'+'/' + link)
            partidos.append(i.text)
    return(links_asistencias_sesion, partidos)


def obtain_links_periodos(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    links_asistencias = []
    for i in soup.findAll("table")[0].findAll('a'):
        h = i.get('href')
        if any(re.findall('votaciones', h)):
            links_asistencias.append(link.split('.gob.mx')[0] + '.gob.mx'+'/' + h)
    return(links_asistencias)



In [137]:
links = obtain_links_periodos(links_legislaturas[0])

In [139]:
links_legislaturas = [
    #'http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_periodonplxiv.php',
    'http://sitllxiii.diputados.gob.mx/votaciones_por_periodonplxiii.php',
    'http://sitllxii.diputados.gob.mx/votaciones_por_periodonplxii.php', 
    'http://sitllxi.diputados.gob.mx/votaciones_por_periodonplxi.php', 
    'http://sitllx.diputados.gob.mx/votaciones_por_periodonp.php'
]

In [140]:
'http://sitl.diputados.gob.mx/LXIV_leg/votacionesxperiodonplxiv.php?pert=1'

'http://sitl.diputados.gob.mx/LXIV_leg/votacionesxperiodonplxiv.php?pert=1'

In [141]:

for link, leg in tqdm(zip(links_legislaturas, [63, 62, 61, 60]), total = 5):
    data_legislatura = []
    links_periodos = obtain_links_periodos(link)
    
    for link2 in tqdm(links_periodos):
        links_sesiones = obtain_links_votaciones(link2)
        for link3 in tqdm(links_sesiones):
            links_sesiones_partido, partidos = obtain_partyVotaciones(link3)
            topic = obtain_topic(link3)
            for link4, p in tqdm(zip(links_sesiones_partido, partidos), total = len(links_sesiones_partido)):
                data_legislatura.append(obtain_votacion(link4, leg, p, topic))
    df_attendance_leg = pd.concat(data_legislatura)
    name_file = 'roll_call' + str(leg) + '.csv'
    df_attendance_leg.to_csv('../../data/02-roll_call_votes/' + name_file, encoding = 'utf-8')

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.33it/s]


 20%|██        | 2/10 [00:00<00:03,  2.25it/s]


 30%|███       | 3/10 [00:01<00:02,  2.64it/s]


 40%|████      | 4/10 [00:01<00:02,  2.89it/s]


 50%|█████     | 5/10 [00:01<00:01,  3.29it/s]


 60%|██████    | 6/10 [00:01<00:01,  3.31it/s]


 70%|███████   | 7/10 [00:02<00:00,  3.85it/s]


 80%|████████  | 8/10 [00:02<00:00,  3.95it/s]


 90%|█████████ | 9/10 [00:02<00:00,  4.42it/s]


100%|██████████| 10/10 [00:02<00:00,  3.79it/s]


  2%|▏         | 1/53 [00:03<02:49,  3.26s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.29it/s]


 20%|██        | 2/10 [00:00<00:03,  2.17it/s]


 30%|███       | 3/10 [00:01<00:02,  2.57it/s]


 40%|████      | 4/10 [00:01<00:02,  2.81it/s]


 50%|█████     | 5/10 [00:01<00:01,  3.06it/s]


 60%|██████    | 6/10 [00:02<00:01,  2.92it/s]


 70%

 90%|█████████ | 9/10 [00:02<00:00,  5.29it/s]


100%|██████████| 10/10 [00:02<00:00,  4.51it/s]


 23%|██▎       | 12/53 [00:33<01:48,  2.66s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:05,  1.58it/s]


 20%|██        | 2/10 [00:01<00:04,  1.70it/s]


 30%|███       | 3/10 [00:01<00:03,  1.95it/s]


 40%|████      | 4/10 [00:01<00:02,  2.10it/s]


 50%|█████     | 5/10 [00:02<00:02,  2.24it/s]


 60%|██████    | 6/10 [00:02<00:01,  2.44it/s]


 70%|███████   | 7/10 [00:02<00:01,  2.70it/s]


 80%|████████  | 8/10 [00:02<00:00,  3.28it/s]


 90%|█████████ | 9/10 [00:03<00:00,  3.86it/s]


100%|██████████| 10/10 [00:03<00:00,  3.03it/s]


 25%|██▍       | 13/53 [00:37<02:01,  3.04s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.82it/s]


 20%|██        | 2/10 [00:00<00:02,  3.02it/s]


 30%|███       | 3/10 [00:00<00:02,  3.38it/s]


 40%|████      | 4/10 [00:01<00:01,  3.66it/s]


 50%|█████     | 5/10 [00:01<00:

 80%|████████  | 8/10 [00:01<00:00,  4.90it/s]


 90%|█████████ | 9/10 [00:02<00:00,  5.33it/s]


100%|██████████| 10/10 [00:02<00:00,  4.54it/s]


 45%|████▌     | 24/53 [01:05<01:15,  2.59s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.73it/s]


 20%|██        | 2/10 [00:00<00:02,  2.92it/s]


 30%|███       | 3/10 [00:00<00:02,  3.31it/s]


 40%|████      | 4/10 [00:01<00:01,  3.65it/s]


 50%|█████     | 5/10 [00:01<00:01,  3.92it/s]


 60%|██████    | 6/10 [00:01<00:00,  4.22it/s]


 70%|███████   | 7/10 [00:01<00:00,  4.66it/s]


 80%|████████  | 8/10 [00:01<00:00,  5.08it/s]


 90%|█████████ | 9/10 [00:01<00:00,  5.53it/s]


100%|██████████| 10/10 [00:02<00:00,  4.78it/s]


 47%|████▋     | 25/53 [01:07<01:11,  2.56s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.67it/s]


 20%|██        | 2/10 [00:00<00:02,  2.86it/s]


 30%|███       | 3/10 [00:00<00:02,  3.24it/s]


 40%|████      | 4/10 [00:01<00:

 70%|███████   | 7/10 [00:01<00:00,  4.09it/s]


 80%|████████  | 8/10 [00:02<00:00,  4.49it/s]


 90%|█████████ | 9/10 [00:02<00:00,  4.88it/s]


100%|██████████| 10/10 [00:02<00:00,  4.17it/s]


 68%|██████▊   | 36/53 [01:40<00:53,  3.17s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.48it/s]


 20%|██        | 2/10 [00:00<00:03,  2.66it/s]


 30%|███       | 3/10 [00:00<00:02,  2.97it/s]


 40%|████      | 4/10 [00:01<00:01,  3.25it/s]


 50%|█████     | 5/10 [00:01<00:01,  3.43it/s]


 60%|██████    | 6/10 [00:01<00:01,  3.67it/s]


 70%|███████   | 7/10 [00:01<00:00,  4.12it/s]


 80%|████████  | 8/10 [00:02<00:00,  4.54it/s]


 90%|█████████ | 9/10 [00:02<00:00,  4.97it/s]


100%|██████████| 10/10 [00:02<00:00,  4.25it/s]


 70%|██████▉   | 37/53 [01:43<00:49,  3.09s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.66it/s]


 20%|██        | 2/10 [00:00<00:02,  2.70it/s]


 30%|███       | 3/10 [00:00<00:

 60%|██████    | 6/10 [00:01<00:01,  3.89it/s]


 70%|███████   | 7/10 [00:01<00:00,  4.34it/s]


 80%|████████  | 8/10 [00:01<00:00,  4.68it/s]


 90%|█████████ | 9/10 [00:02<00:00,  4.99it/s]


100%|██████████| 10/10 [00:02<00:00,  4.39it/s]


 91%|█████████ | 48/53 [02:14<00:14,  2.87s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:03,  2.68it/s]


 20%|██        | 2/10 [00:00<00:02,  2.85it/s]


 30%|███       | 3/10 [00:00<00:02,  3.14it/s]


 40%|████      | 4/10 [00:01<00:01,  3.41it/s]


 50%|█████     | 5/10 [00:01<00:01,  3.65it/s]


 60%|██████    | 6/10 [00:01<00:01,  3.88it/s]


 70%|███████   | 7/10 [00:01<00:00,  4.30it/s]


 80%|████████  | 8/10 [00:01<00:00,  4.67it/s]


 90%|█████████ | 9/10 [00:02<00:00,  5.00it/s]


100%|██████████| 10/10 [00:02<00:00,  4.39it/s]


 92%|█████████▏| 49/53 [02:17<00:11,  2.85s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:00<00:04,  2.21it/s]


 20%|██        | 2/10 [00:00<00:

...

 89%|████████▉ | 8/9 [00:02<00:00,  4.73it/s]


100%|██████████| 9/9 [00:02<00:00,  3.93it/s]


 61%|██████    | 92/151 [04:40<02:59,  3.04s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:05,  1.54it/s]


 22%|██▏       | 2/9 [00:01<00:04,  1.75it/s]


 33%|███▎      | 3/9 [00:01<00:03,  1.99it/s]


 44%|████▍     | 4/9 [00:01<00:02,  2.49it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.03it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.60it/s]


 78%|███████▊  | 7/9 [00:02<00:00,  4.10it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.57it/s]


100%|██████████| 9/9 [00:02<00:00,  3.83it/s]


 62%|██████▏   | 93/151 [04:43<02:52,  2.98s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:04,  1.97it/s]


 22%|██▏       | 2/9 [00:00<00:03,  2.13it/s]


 33%|███▎      | 3/9 [00:01<00:02,  2.32it/s]


 44%|████▍     | 4/9 [00:01<00:01,  2.85it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.36it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.90it/s]


 7

 70%|██████▉   | 105/151 [05:17<02:13,  2.90s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:03,  2.02it/s]


 22%|██▏       | 2/9 [00:00<00:03,  2.15it/s]


 33%|███▎      | 3/9 [00:01<00:02,  2.33it/s]


 44%|████▍     | 4/9 [00:01<00:01,  2.61it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.00it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.52it/s]


 78%|███████▊  | 7/9 [00:02<00:00,  4.02it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.52it/s]


100%|██████████| 9/9 [00:02<00:00,  3.78it/s]


 70%|███████   | 106/151 [05:20<02:09,  2.88s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:04,  1.92it/s]


 22%|██▏       | 2/9 [00:01<00:03,  1.84it/s]


 33%|███▎      | 3/9 [00:01<00:02,  2.07it/s]


 44%|████▍     | 4/9 [00:01<00:01,  2.59it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.12it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.65it/s]


 78%|███████▊  | 7/9 [00:02<00:00,  4.18it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.66it/s]




 11%|█         | 1/9 [00:00<00:04,  1.63it/s]


 22%|██▏       | 2/9 [00:01<00:04,  1.73it/s]


 33%|███▎      | 3/9 [00:01<00:03,  1.99it/s]


 44%|████▍     | 4/9 [00:01<00:02,  2.48it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.03it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.58it/s]


 78%|███████▊  | 7/9 [00:02<00:00,  4.13it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.65it/s]


100%|██████████| 9/9 [00:02<00:00,  3.75it/s]


 79%|███████▉  | 119/151 [05:58<01:31,  2.85s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:04,  1.99it/s]


 22%|██▏       | 2/9 [00:00<00:03,  2.09it/s]


 33%|███▎      | 3/9 [00:01<00:02,  2.28it/s]


 44%|████▍     | 4/9 [00:01<00:01,  2.80it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.21it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.73it/s]


 78%|███████▊  | 7/9 [00:01<00:00,  4.28it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.75it/s]


100%|██████████| 9/9 [00:02<00:00,  3.96it/s]


 79%|███████▉  | 120/151 [06:01<01:27,  2.83

 33%|███▎      | 3/9 [00:01<00:02,  2.31it/s]


 44%|████▍     | 4/9 [00:01<00:01,  2.86it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.40it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.92it/s]


 78%|███████▊  | 7/9 [00:01<00:00,  4.42it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.90it/s]


100%|██████████| 9/9 [00:02<00:00,  4.13it/s]


 87%|████████▋ | 132/151 [06:42<01:08,  3.58s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:04,  1.94it/s]


 22%|██▏       | 2/9 [00:00<00:03,  2.11it/s]


 33%|███▎      | 3/9 [00:01<00:02,  2.31it/s]


 44%|████▍     | 4/9 [00:01<00:01,  2.54it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.08it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.64it/s]


 78%|███████▊  | 7/9 [00:02<00:00,  4.18it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.68it/s]


100%|██████████| 9/9 [00:02<00:00,  3.87it/s]


 88%|████████▊ | 133/151 [06:45<01:00,  3.34s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:03,  2.02it/s]




 56%|█████▌    | 5/9 [00:01<00:01,  2.99it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.58it/s]


 78%|███████▊  | 7/9 [00:02<00:00,  4.13it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.66it/s]


100%|██████████| 9/9 [00:02<00:00,  3.69it/s]


 96%|█████████▌| 145/151 [07:21<00:18,  3.01s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:04,  1.81it/s]


 22%|██▏       | 2/9 [00:00<00:03,  1.98it/s]


 33%|███▎      | 3/9 [00:01<00:02,  2.19it/s]


 44%|████▍     | 4/9 [00:01<00:01,  2.70it/s]


 56%|█████▌    | 5/9 [00:01<00:01,  3.20it/s]


 67%|██████▋   | 6/9 [00:01<00:00,  3.70it/s]


 78%|███████▊  | 7/9 [00:01<00:00,  4.19it/s]


 89%|████████▉ | 8/9 [00:02<00:00,  4.71it/s]


100%|██████████| 9/9 [00:02<00:00,  3.95it/s]


 97%|█████████▋| 146/151 [07:24<00:14,  2.96s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:00<00:03,  2.03it/s]


 22%|██▏       | 2/9 [00:00<00:03,  2.10it/s]


 33%|███▎      | 3/9 [00:01<00:02,  2.31it/s]


